In [2]:
import pandas as pd
import numpy as np
import pickle
import treecorr
from tqdm import tqdm_notebook as tqdm

In [3]:
df_all=[None]*4


In [4]:
for i in range(4):
    df_all[i]=pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(i+2))

In [5]:
df_riz = pd.read_pickle('/home/cz136/project/balrog/data/mcal{}_df_all.pkl'.format(3)) 

In [6]:
keys = df_riz.columns[40:]

In [7]:
true_df=pd.read_pickle("/home/cz136/project/balrog/data/true_df.pkl")
mcal_list = [df_all]
df_riz_list= [df_riz]

In [11]:
def add_tilename(df_list,true):
    for i in range(4):
        df_list[i]=pd.concat([df_list[i],true_df[['meas_tilename']]],axis=1,join='inner')

In [8]:
for df_list in mcal_list:
    add_tilename(df_list,true_df)

In [9]:
df_riz=pd.concat([df_riz,true_df[['true_tilename']]],axis=1,join='inner')

In [10]:
def get_ke(catalog,key):
    delta_g1=catalog['e_1'].to_numpy()
    delta_g2=catalog['e_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    return(kg)

In [11]:
def get_ke_true(catalog,key):
    delta_g1=catalog['true_cm_g_1'].to_numpy()
    delta_g2=catalog['true_cm_g_2'].to_numpy()
    k=catalog[key].to_numpy()
    ra=catalog['ra'].to_numpy()
    dec=catalog['dec'].to_numpy()
    
    kcat= treecorr.Catalog( k=k,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    gcat = treecorr.Catalog( g1=delta_g1, g2=delta_g2,
                                 ra=ra, dec=dec, 
                                 ra_units='deg', dec_units='deg')
    
    kg = treecorr.KGCorrelation(nbins=20, min_sep=2.5, max_sep=250, sep_units='arcmin', bin_slop=0.2)
    kg.process(kcat,gcat)
    return(kg)

In [12]:
def jk_resample(df,key):
    tile_list=df['true_tilename'].unique()
    jk_ke=[None]*len(tile_list)
    jk_ke_true=[None]*len(tile_list)
    for i,tile in tqdm(enumerate(tile_list)):
        jk_sample=df[np.logical_not((df['true_tilename']==tile))]  
        jk_ke[i]=get_ke(jk_sample,key)
        jk_ke_true[i]=get_ke_true(jk_sample,key)
    return(jk_ke,jk_ke_true)

In [13]:
# jk_positive=[jk_resample(df,key1) for df in df_positive]
# print ("Positive mcal jackknifed.")
# jk_negative=[jk_resample(df,key1) for df in df_negative]
# print ("Negative mcal jackknifed.")
jk_all_1 = jk_resample(df_all[0],key1) 
# print ("Unselected mcal jackknifed.")

NameError: name 'key1' is not defined

In [ ]:
jk_all_2=jk_resample(df_all[1],key1)

In [ ]:
jk_all_3 = jk_resample(df_all[0],key2)

In [ ]:
jk_all_4 = jk_resample(df_all[1],key2) 

In [ ]:
jk_all_5= jk_resample(df_all[0],key3)  
jk_all_6= jk_resample(df_all[1],key3)  

In [30]:
def pickler(df_list,head):
    for i,df in enumerate(df_list):
        df.to_pickle("/home/cz136/project/balrog/data/jackknife/{}_{}.pkl".format(head,i))

In [31]:
def get_jkerror(ke_jk_sample):
    n=len(ke_jk_sample)
    xi_array=np.array([jk.xi for jk in ke_jk_sample])
    x_array = np.mean([np.exp(jk.meanlogr) for jk in ke_jk_sample], axis=0) 
    emp_average=1/n*np.sum(xi_array,axis=0)
    jk_se= np.sqrt((n-1)/n * np.sum((xi_array - emp_average)**2,axis=0))
    
    return(emp_average, jk_se,x_array)

In [32]:
def get_cov(ke_pair):
    ke,ke_true=ke_pair
    n=len(ke)
    xi_array=np.array([jk.xi for jk in ke])
    xi_true_array=np.array([jk.xi for jk in ke_true])
    diff_array=xi_array-xi_true_array
    emp_average=1/n*np.sum(diff_array,axis=0)
    nd = len(diff_array[0])
    cov=np.empty([nd,nd])
    for row in range(nd):
        for column in range(nd):
            cov[row][column]=\
            (n-1)/n * np.sum((diff_array[:,row]-emp_average[row])*(diff_array[:,column]-emp_average[column]))
    return (cov)

In [33]:
def get_diff_error(ke,ke_true):
    n=len(ke)
    if len(ke)!=len(ke_true):
        print ('Length error')
    ke_xi_array = np.array([corr.xi for corr in ke])
    ke_true_xi_array = np.array([corr.xi for corr in ke_true])
    xi_array = ke_xi_array - ke_true_xi_array
    emp_average=1/n*np.sum(xi_array,axis=0)
    jk_se= np.sqrt((n-1)/n * np.sum((xi_array - emp_average)**2,axis=0))
    return (jk_se)

In [34]:
def get_chisqr(covariance,y_value):
    return(np.dot(np.dot(y_value,np.linalg.inv(covariance)),y_value))

In [ ]:
jk_array=[jk_all_1,jk_all_2,jk_all_3,jk_all_4]
error_array=[get_diff_error(*jk_all) for jk_all in jk_array]

In [35]:
def process_pair(jk_tuple):
    (ke,ke_true)=jk_tuple
    (ke_average,ke_error, ke_x)=get_jkerror(ke)
    (ke_true_average,ke_true_error,ke_true_x)=get_jkerror(ke_true)
    (a,b,error)=get_jkerror(ke-ke_true)
    diff=np.absolute(ke_average-ke_true_average)
#     error=ke_error + ke_true_error
    return (diff,error,ke_x)

In [36]:
covariance_array = [get_cov(jk_all) for jk_all in jk_array]
chisqr_array = [None]*4

NameError: name 'jk_array' is not defined

In [20]:
import matplotlib.pyplot as plt



def plot_error(error,key,i):
    
    mof_array=['MOF','woMOF']
    
    if i==0 or i==2:
        j=0
    else:
        j=1
        
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)


    all_ke = get_ke(df_all[j],key)
    all_ke_true=get_ke_true(df_all[j],key)
    
    x_val=np.exp(all_ke.meanlogr)
    y_val = np.absolute(all_ke_true.xi-all_ke.xi)
    
    chisqr_array[i]=get_chisqr(covariance_array[i],y_val)
    
    fig, ax = plt.subplots()

    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(r'$\delta \xi = \xi_{{measured}} - \xi_{{true}}$, {}, {}'.format(key,mof_array[j]))

    ax.errorbar(x=x_val,y=y_val,yerr=error,ls='none')
    ax.text(0.85, 0.95, "$\chi^2={:.2f}$".format(chisqr_array[i]), transform=ax.transAxes, fontsize=7,
verticalalignment='top', bbox=props)
    ax.plot(x_val,y_val)
    plt.savefig("/home/cz136/project/balrog/{}.png".format(i),dpi=500)
    plt.close()

# plt.show()

In [ ]:
plot_error(error_array[0],key1,0)
plot_error(error_array[1],key1,1)
plot_error(error_array[2],key2,2)
plot_error(error_array[3],key2,3)

In [ ]:
f = open("demofile3.txt", "w")
f.write("Woops! I have deleted the content!")
f.close()

In [37]:
def get_chisqr_for_key(key):
    
    jk_sample=jk_resample(df_riz,key)
    cov = get_cov(jk_sample)
    
    all_ke=get_ke(df_riz,key)
    all_ke_true=get_ke_true(df_riz,key)
    
    y_value=all_ke_true.xi-all_ke.xi
    
    chi_sqr=get_chisqr(cov,y_value)
    
    return(chi_sqr)

In [39]:
f = open("/home/cz136/project/balrog/chisqr.txt","w")
for key in keys:
    print(key)
    chi_sqr=get_chisqr_for_key(key)
    print(chi_sqr)
    f.write("chi**2_{}={}".format(key,chi_sqr))

ebv_planck13


5.503516173054164
star_density_piffv23


7.910286246733245
det_frac


9.005985232622582
('g', 'airmass')


9.123005667667078
('g', 'fwhm')


...

8.860773294837728
('z', 'sigma_zp')


8.226556890082815
('z', 'sky_var')


9.322254953138252
('z', 'teff_mean')


8.746757757740552
('z', 'teff_sum')


8.237298597191927
